In [1]:
import tensorflow as tf
from tensorflow import keras
import sklearn

import numpy as np
import pandas as pd
import os

np.random.seed(42)
tf.random.set_seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.style.use('seaborn-darkgrid')

# Tensor

### tensor & arithmetic operation

In [ ]:
print(tf.constant([[1.,2.,3.], [4.,5.,6.]])) # matrix
print(tf.constant(42)) # scalar

In [ ]:
t = tf.constant([[1.,2.,3.], [4.,5.,6.]])
print(t.shape)
print(t.dtype)

In [ ]:
print(t[:,1:])
print(t[...,1])
print(t[...,1,tf.newaxis])

In [ ]:
print(t+10)
print(t-1)
print(t*2)
print(tf.square(t))
print(t@tf.transpose(t))

### tensor & numpy

In [ ]:
a = np.array([2., 4., 5.])
print(tf.constant(a),'\n')
print(t.numpy(),'\n')
print(tf.square(a),'\n')
print(np.square(t))

In [ ]:
tf.constant(a, dtype = tf.float32)

### type transformation

In [ ]:
# tf.constant(2.)+tf.constant(40) # error!

In [ ]:
# tf.constant(2.)+tf.constant(40., dtype = tf.float64) # error!

In [ ]:
t2 = tf.constant(40., dtype = tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

### variable

In [ ]:
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])
v

In [ ]:
v.assign(2*v)
print(v)
v[0,1].assign(42)
print(v)
v[:,2].assign([0.,1.])
print(v)
v.scatter_nd_update(indices = [[0,0],[1,2]], updates = [100., 200.])
print(v)

# user define Model & Algorithm

### user define loss function

In [2]:
# Huber loss (with fixed threshold : 1)
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) -0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [3]:
# data preparing
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# model building
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

# fitting (using Huber loss)
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])
model.fit(X_train_scaled, y_train, epochs=3,
          validation_data=(X_valid_scaled, y_valid))

### saving & loading models with custom elements 

In [ ]:
# model save
model.save("my_model_with_a_custom_loss.h5")

In [ ]:
# model load
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

In [ ]:
# set threshold 
def create_huber(threshold = 1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [ ]:
model.compile(loss = create_huber(2.0), optimizer = "nadam")
model.fit(X_train_scaled, y_train, epochs=3,
          validation_data=(X_valid_scaled, y_valid))

In [ ]:
# save & load
# 임계값은 함께 저장되지 않는다.
model.save("my_model_with_a_custom_loss_threshold_2.h5")
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [ ]:
# 대안으로 아래와 같이 새로운 클래스를 정의하여 임계값까지 저장되도록 할 수 있음
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_error = tf.square(error) / 2
        linear_error = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        # 부모 클래스의 config에 새로운 하이퍼파라미터 threshold를 추가하여 반환
        return {**base_config, "threshold":self.threshold}

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

model.save("my_model_with_a_custom_loss_class.h5")
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})

#### activation/initializing/regularization/constraint custumizing

In [ ]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype = tf.float32):
    stddev = tf.sqrt(2. / (shape[0]+shape[1]))
    return tf.random.normal(shape, stddev = stddev, dtype = dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01*weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [ ]:
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

In [ ]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                       kernel_regularizer=MyL1Regularizer(0.01))
])

### user define metrics

In [4]:
# streaming metrics
precision = keras.metrics.Precision()

# user define streaming metrics
class HuberMetric(keras.metrics.Mean):
    def __init__(self, threshold=1.0, name='HuberMetric', dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        super(HuberMetric, self).update_state(metric, sample_weight)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}        

### user define layer  
####   
동일한 층 블럭이 여러 번 반복되는 네트워크 (a,b,c,a,b,c,a,b,c) 를 만들 경우  
각각의 층 블럭(a,b,c)을 하나의 사용자 정의 층(d)으로 정의하고 (d,d,d)와 같이 만들 수 있다.

In [ ]:
# 가중치/하이퍼파라미터가 필요 없는 사용자 정의 층
# 파이썬 함수를 만든 후 keras.layers.Lambda 층으로 감싼다.
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [ ]:
# 상태가 있는 사용자 정의 층 (Dense층 간소화버전 구현하기)
class MyDense(keras.layers.Layer):
    # 기본적인 요소들은 모두 상속. 층의 뉴런수와 활성화함수를 별도의 특성으로 가짐
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    # shape에 맞게 층의 가중치를 생성
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end
    # 실제로 층에서 수행하는 연산
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

In [6]:
# 입력 / 출력 차원수의 확장 (INPUT : 2, OUTPUT : 3)
# 함수형 API, 서브클래싱 API에서만 사용가능
class MyMultiLayer(keras.layers.Layer):
    def call(self, X): # X는 튜플
        X1, X2 = X
        return [X1 + X2, X1 * X2, X1 / X2]
    
    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        return [b1,b1,b1]

In [8]:
# 훈련단계와 테스트단계에서 다르게 동작하는 층
# -> call() 메서드에 bool값을 받는 training 매개변수를 추가하자
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
    
    def call(self, X, training = None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev = self.stddev)
            return X + noise
        else:
            return X
    
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

### user define model

In [ ]:
# skip connection 이 있는 사용자 정의 residual block 만들기 (자기 순회가 가능하도록)
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation = "elu",
                                          kernel_initializer = "he_normal")
                       for _ in range(n_layers)]
        def call(self, inputs):
            Z = inputs
            for layer in self.hidden:
                Z = layer(Z)
            return inputs + Z

In [ ]:
# subclassing API 모델 정의
class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation = "elu",
                                          kernel_initializer = "he_normal")
        self.block1 = ResidualBlock(2,30)
        self.block2 = ResidualBlock(2,30)
        self.out = keras.layers.Dense(output_dim)
    
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

### loss based on model components 

In [10]:
# 재구성 손실을 갖는 모델
class ReconstructingRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs, name='recon')
        super().build(batch_input_shape)

    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        self.recon_loss = 0.05 * tf.reduce_mean(tf.square(reconstruction - inputs))
        return self.out(Z)

### calculate gradient via auto-diff

In [12]:
# sample function
def f(w1,w2):
    return 3*w1**2 + 2*w1*w2

w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [13]:
with tf.GradientTape(persistent = True) as tape:
    z = f(w1, w2)

dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2)
del tape

In [14]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

# Tensorflow functions and graph    
  
  
텐서플로 함수는 적절한 순서에 맞춰 계산그래프 내의 연산을 효율적으로 실행한다.  
(원본 파이썬 함수보다 훨씬 빠르게 실행된다.)  
-> 파이썬 함수를 빠르게 실행하려면 텐서플로 함수로 변환하는 것이 좋다.

In [16]:
# sample function
def cube(x):
    return x**3

print(cube(2))
print(cube(tf.Variable(2.)))

8
tf.Tensor(8.0, shape=(), dtype=float32)


In [21]:
tf_cube = tf.function(cube)
print(tf_cube(2))
print(tf_cube(tf.Variable(2.)))

tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(8.0, shape=(), dtype=float32)


내부적으로 tf.function()은 cube()함수에서 수행되는 계산을 분석하고  
동일한 작업을 수행하는 계산 그래프를 생성한다.  
  
다른 방법으로 tf.function 데코레이터를 사용할 수 있다.

In [22]:
@tf.function
def tf_cube(x):
    return x**3